# Dataloader

In [3]:
import os
os.chdir('..')
from utils.dataloader import GANDataset, GenDataIter

In [5]:
data_path = r"D:\diploma\github\dpl\data" #\real
BATCH_SIZE = 64

In [7]:
iterator = GenDataIter(data=data_path,
                            batch_size=BATCH_SIZE,
                            shuffle=None)

In [8]:
loader = iterator.loader
loader

In [9]:
for el in loader:
    print(el)
    x = el['input']
    
    for token in x:
        print([iterator.idx2word[str(l.item())] for l in token])
        break
    break

{'input': tensor([[   1, 5171, 5852,  ...,    0,    0,    0],
        [   1,  100, 5485,  ..., 5554, 1402,    0],
        [   1, 3585,  830,  ...,    0,    0,    0],
        ...,
        [   1,  141,  736,  ...,    0,    0,    0],
        [   1, 3620, 3759,  ...,    0,    0,    0],
        [   1, 3585, 5806,  ...,    0,    0,    0]]), 'target': tensor([[5171, 5852, 4559,  ...,    0,    0,    0],
        [ 100, 5485, 5449,  ..., 1402,    0,    0],
        [3585,  830, 4956,  ...,    0,    0,    0],
        ...,
        [ 141,  736, 3026,  ...,    0,    0,    0],
        [3620, 3759,  103,  ...,    0,    0,    0],
        [3585, 5806,  698,  ...,    0,    0,    0]])}
['<start>', 'when', 'it', 'comes', 'to', 'management', ',', 'i', '’', 've', 'always', 'been', 'a', 'bigger', 'believer', 'in', 'fundamentals', 'than', 'fancy', '.', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>',

In [10]:
iterator.word2idx['<pad>']

'0'

In [11]:
iterator.idx2word['1'], iterator.idx2word['5171'], iterator.idx2word['5852']

('<start>', 'when', 'it')

# Generator

In [1]:
import os
os.chdir('..')

In [5]:
import torch
from models.RelGAN_G import RelGAN_G
MAX_SEQ_LEN = 50
PAD_IDX = 0
VOCAB_SIZE = 6330
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if_cuda = True if torch.cuda.is_available() else False

In [6]:
gen = RelGAN_G(mem_slots=1, num_heads=2, head_size=256, embedding_dim=32, hidden_dim=32,
                            vocab_size=VOCAB_SIZE, max_seq_len=MAX_SEQ_LEN, padding_idx=PAD_IDX, gpu=if_cuda)

# Discriminator

In [9]:
from models.RelGAN_D import RelGAN_D
dis = RelGAN_D(embed_dim=64, 
               max_seq_len=MAX_SEQ_LEN,
               num_rep=64, 
               vocab_size=VOCAB_SIZE, 
               padding_idx=PAD_IDX, 
               weights = 'uniform',
               gpu=if_cuda, 
               dropout=0.25)

# Instructor

In [1]:
import os
os.chdir('..')
from models.instructor import RelGANInstructor

In [2]:
instructor = RelGANInstructor()

# Checking

In [3]:
instructor._run()

Starting GENERATOR MLE TRAINING
[MLE-GEN] epoch 0 : pre_loss = 5.4460, BLEU-[2, 3, 4, 5] = [0.403, 0.131, 0.077, 0.057], NLL_gen = 3.2112, NLL_div = 2.9414, Self-BLEU-[2, 3, 4] = [0.224, 0.084, 0.057]
[MLE-GEN] epoch 1 : pre_loss = 2.7335, BLEU-[2, 3, 4, 5] = [0.653, 0.343, 0.179, 0.112], NLL_gen = 1.9271, NLL_div = 1.9373, Self-BLEU-[2, 3, 4] = [0.336, 0.138, 0.077]


KeyboardInterrupt: 

In [4]:
instructor.adv_train_generator(1)

0.6930944919586182

In [5]:
instructor.adv_train_discriminator(5)

0.6930819153785706

In [4]:
# instructor.gen.sample()
from utils.dataloader import GANDataset, GenDataIter
from utils.preprocess import *
SAMPLES_NUM = 100
BATCH_SIZE = 64

eval_samples = instructor.gen.sample(SAMPLES_NUM, 4 * BATCH_SIZE)
# gen_data = GenDataIter(eval_samples, batch_size=BATCH_SIZE)
# gen_tokens = tensor_to_tokens(eval_samples, instructor.idx2word_dict)
# gen_tokens_s = tensor_to_tokens(instructor.gen.sample(200, 200), instructor.idx2word_dict)

# # Reset metrics
# # self.bleu.reset(test_text=gen_tokens, real_text=instructor.test_data.tokens)
# instructor.nll_gen.reset(instructor.gen, instructor.train_data.loader)
# instructor.nll_div.reset(self.gen, gen_data.loader)
# instructor.self_bleu.reset(test_text=gen_tokens_s, real_text=gen_tokens)
# instructor.ppl.reset(gen_tokens)

In [7]:
gen_tokens = tensor_to_tokens(eval_samples, instructor.idx2word_dict)

In [10]:
for el in gen_tokens:
    print(el)
    print()

['’', 'second-', 'racing', 'of', 'our', 'mayor', 'bullets', 'oil', 'prices', 'weighed', ',', 'and', 'he', 'found', 'of', 'this', 'over', 'a', 'pep', 'talk', 'interviews', '4', 'at', 'a', 'record', 'tribe', 'was', 'driven', 'service', 'is', 'sent', 'of', 'making', 'potash', '.']

['happy', 'the', 'democratic', 'murder', 'left', '``', 'the', 'tricks', 'don', 'hit', '.']

['``', 'these', 'youngsters', '’', 's', 'weekend', '.']

['the', 'newspaper', 'zoo', 'it', 'is', 'accused', "'s", "'s", 'right', '.']

['jul', '4', '4', ',', 'all', 'of', 'the', 'game', 'was', 'hit', 'hard', '.']

['rowe', 'the', 'powerful', 'is', 'up', 'this', 'was', 'beaten', 'affair', 'by', ',', 'screaming', 'when', 'it', 'was', '225', 'found', 'for', 'biting', 'treat', '.']

['the', 'dow', 'jones', 'industrial', 'unpredictable', 'up', 'the', 'hearing', 'ended', 'site', 'will', 'be', 'of', 'illustrative', ',', 'joined', 'if', ',', 'officials', ',', 'would', 'to', 'be', 'up', 'the', 'situation', 'grill', '.']

['routin